In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import resnet_model
import os

%matplotlib inline

/home/barney_awesome_li/anaconda3/envs/tfgpu/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [3]:
# Load training data set
train = pd.read_json('../data/train.json')

In [4]:
print(train.head())

                                              band_1  \
0  [-27.878360999999998, -27.15416, -28.668615, -...   
1  [-12.242375, -14.920304999999999, -14.920363, ...   
2  [-24.603676, -24.603714, -24.871029, -23.15277...   
3  [-22.454607, -23.082819, -23.998013, -23.99805...   
4  [-26.006956, -23.164886, -23.164886, -26.89116...   

                                              band_2        id inc_angle  \
0  [-27.154118, -29.537888, -31.0306, -32.190483,...  dfd5f913   43.9239   
1  [-31.506321, -27.984554, -26.645678, -23.76760...  e25388fd   38.1562   
2  [-24.870956, -24.092632, -20.653963, -19.41104...  58b2aaa0   45.2859   
3  [-27.889421, -27.519794, -27.165262, -29.10350...  4cfc3a18   43.8306   
4  [-27.206915, -30.259186, -30.259186, -23.16495...  271f93f4   35.6256   

   is_iceberg  
0           0  
1           0  
2           1  
3           0  
4           0  


In [5]:
order_num = np.random.rand(train.shape[0])
train['order'] = order_num
train.sort_values('order', axis=0, inplace=True)
print(train.head())

                                                 band_1  \
517   [-22.912132, -22.103603, -23.572226, -24.04189...   
607   [-16.1071, -19.369699, -22.577993, -20.602985,...   
1219  [-24.44771, -21.25375, -19.315607, -18.671972,...   
406   [-16.044106, -14.972523, -13.678087, -16.3116,...   
720   [-17.194717, -14.107332, -13.673001, -14.64330...   

                                                 band_2        id inc_angle  \
517   [-24.538231, -26.224693, -28.319445, -30.06249...  7a665fa2   43.2697   
607   [-28.415577, -27.076717, -24.655916, -22.39520...  f09265ef   34.0171   
1219  [-26.031334, -25.203537, -25.746641, -31.49153...  390938d4   38.4741   
406   [-22.892561, -21.471365, -23.0879, -24.830975,...  a63467f2   34.8705   
720   [-24.10696, -24.589689, -26.84383, -22.801788,...  f10fcdd5   39.6622   

      is_iceberg     order  
517            0  0.000305  
607            1  0.001021  
1219           1  0.001623  
406            0  0.002420  
720            0  0.00255

In [6]:
raw_y = train['is_iceberg'].tolist()
tensor_y = tf.one_hot(raw_y, depth=2)
with tf.Session() as sess:
    total_y = np.array(sess.run(tensor_y))
print(total_y.shape)

(1604, 2)


In [7]:
raw_x_1 = np.array(train['band_1'].tolist())
raw_x_2 = np.array(train['band_2'].tolist())
total_x = np.concatenate((raw_x_1, raw_x_2), axis = 1)
total_x = raw_x_1
print(total_x.shape)

(1604, 5625)


In [8]:
dev_x = total_x[0:300,:]
dev_y = total_y[0:300,:]
print(dev_x.shape)
train_x = total_x[300:,:]
train_y = total_y[300:,:]
print(train_x.shape)
print(train_y.shape)

(300, 5625)
(1304, 5625)
(1304, 2)


In [9]:
class MiniBatch(object):
    def __init__(self, x, y, batch_size):
        self.generator = self.slice_random(x, y, batch_size)
    
    def next_batch(self):
        try:
            return(next(self.generator))
        except StopIteration as e:
            return None, None
        
    @staticmethod    
    def slice_random(X, Y, mini_batch_size):
        tmp_X = X.copy()
        tmp_Y = Y.copy()
        cur_index = 0
        while(tmp_X.shape[0]>0):
            pick_size = tmp_X.shape[0] if tmp_X.shape[0]<mini_batch_size else mini_batch_size
            mini_batch_x = tmp_X[cur_index:cur_index + pick_size,:]
            mini_batch_y = tmp_Y[cur_index:cur_index + pick_size,:]
            cur_index += pick_size
            yield mini_batch_x, mini_batch_y

In [10]:
# test
test_mini_batch = MiniBatch(dev_x, dev_y, 50)


In [11]:
with tf.name_scope('reshape'):
    x_in = tf.placeholder(tf.float32)
    y_in = tf.placeholder(tf.float32)
    x = tf.image.resize_image_with_crop_or_pad(tf.reshape(x_in, [-1,75,75,1]), 256, 256) 
    y = tf.reshape(y_in, [-1, 2])

In [12]:
with tf.name_scope('resnet'):
    y_generator = resnet_model.imagenet_resnet_v2(18, 2, 'channels_last')
    y_ = y_generator(x, True)

In [13]:
with tf.name_scope('cost'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_, labels=y))

In [14]:
with tf.name_scope('optimizer'):
    optimizer = tf.train.AdamOptimizer().minimize(cost)

In [15]:
with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_,1))
    correct_prediction = tf.cast(correct_prediction, tf.float32)
    accuracy = tf.reduce_mean(correct_prediction)

In [16]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    tf.add_to_collection('accuracy', accuracy)
    tf.add_to_collection('y_', y_)
    tf.add_to_collection('x_in', x_in)
    tf.add_to_collection('y_in', y_in)
    # writer = tf.summary.FileWriter('./logs',sess.graph)
    for i in range(1001):
        minibatch = MiniBatch(train_x, train_y, 64)
        x_batch,y_batch = minibatch.next_batch()
        if x_batch is None or x_batch.shape[0]<64:
            minibatch = MiniBatch(train_x, train_y, 64)
            x_batch,y_batch = minibatch.next_batch()
        # print('minibatch size {} and {}'.format(x_batch.shape, y_batch.shape))
        if i%100 == 0:
            testMiniBatch = MiniBatch(dev_x, dev_y, 64)
            x_batch_test, y_batch_test = testMiniBatch.next_batch()
            if x_batch_test is None or x_batch_test.shape[0]<64:
                testMiniBatch = MiniBatch(dev_x, dev_y, 64)
                x_batch_test, y_batch_test = testMiniBatch.next_batch()
            train_accuracy = accuracy.eval(session=sess,
                                            feed_dict={
                                                x_in:x_batch_test, y_in:y_batch_test
                                            })
            print('step {}, training accuracy {}'.format(i, train_accuracy))
            print('test accuracy {}'.format(accuracy.eval(session=sess, feed_dict={x_in: dev_x , y_in: dev_y})))
            saver = tf.train.Saver()
            saved_path = saver.save(sess, './models/{}.ckpt'.format(i))
            print('model saved to {}'.format(saved_path))
        sess.run(optimizer, feed_dict={x_in:x_batch, y_in:y_batch})
    # writer.close()
    print('test accuracy {}'.format(accuracy.eval(session=sess, feed_dict={x_in: dev_x , y_in: dev_y})))
    

step 0, training accuracy 0.421875
test accuracy 0.4699999988079071
model saved to ./models/0.ckpt
step 100, training accuracy 0.609375
test accuracy 0.5766666531562805
model saved to ./models/100.ckpt
step 200, training accuracy 0.609375
test accuracy 0.5699999928474426
model saved to ./models/200.ckpt
step 300, training accuracy 0.609375
test accuracy 0.5733333230018616
model saved to ./models/300.ckpt
step 400, training accuracy 0.609375
test accuracy 0.5699999928474426
model saved to ./models/400.ckpt
step 500, training accuracy 0.609375
test accuracy 0.5666666626930237
model saved to ./models/500.ckpt
step 600, training accuracy 0.625
test accuracy 0.5633333325386047
model saved to ./models/600.ckpt
step 700, training accuracy 0.625
test accuracy 0.5666666626930237
model saved to ./models/700.ckpt
step 800, training accuracy 0.625
test accuracy 0.5666666626930237
model saved to ./models/800.ckpt
step 900, training accuracy 0.625
test accuracy 0.5666666626930237
model saved to ./mo

In [17]:
# with tf.Session() as sess:
#     saver = tf.train.import_meta_graph('./models/1st.ckpt.meta')
#     saver.restore(sess, './models/1st.ckpt')
#     accuracy = tf.get_collection('accuracy')[0]
#     print('model restored')
#     print('test accuracy {}'.format(accuracy.eval(session=sess, feed_dict={x_in:mnist.test.images, y_in:mnist.test.labels})))

In [18]:
# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     writer = tf.summary.FileWriter('../logs',sess.graph)
#     accuracy_res = accuracy.eval(session=sess, feed_dict={x_in:train_x[0:10], y_in:train_y[0:10]})
#     print(accuracy_res)
#     writer.close()